<a href="https://colab.research.google.com/github/drf0rk/AnxLight/blob/main/notebook/AnxLight_Launcher_v0.1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Cell 1: AnxLight v3 Pre-Flight Setup (ALWAYS Force Clean Clone)
# This cell ALWAYS deletes the old repository to ensure the latest version is used.
# This is the definitive fix for environment caching issues.

import os
import subprocess
import sys
import shutil
from datetime import datetime

# --- Configuration ---
repo_path = '/content/AnxLight'

# --- Unified Logging Setup ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = f"/content/anxlight_logs_{timestamp}"
os.makedirs(log_dir, exist_ok=True)
session_log_file = f"{log_dir}/session_log_{timestamp}.txt"
os.environ['ANXLIGHT_LOG_FILE'] = session_log_file

def log_message(message, level="INFO"):
    timestamp_msg = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_msg = f"[{timestamp_msg}] [{level}] {message}"
    print(formatted_msg)
    with open(session_log_file, "a") as f:
        f.write(formatted_msg + "\n")

log_message("--- CELL 1: PRE-FLIGHT SETUP (ALWAYS FORCING CLEAN CLONE) ---")

# --- Step 1: ALWAYS Delete and Re-Clone ---
if os.path.exists(repo_path):
    log_message(f"Removing existing directory at {repo_path} to ensure a clean state...", "WARNING")
    try:
        shutil.rmtree(repo_path)
        log_message("Directory removed successfully.")
    except Exception as e:
        log_message(f"Error removing directory: {e}. Please check permissions.", "ERROR")
        sys.exit(1)

log_message(f"Cloning fresh AnxLight repository to {repo_path}...")
try:
    subprocess.run(
        ['git', 'clone', 'https://github.com/drf0rk/AnxLight.git', repo_path],
        check=True, capture_output=True, text=True, timeout=120
    )
    log_message("Fresh clone successful.")
except Exception as e:
    log_message(f"FATAL: Failed to clone repository. Error: {e}", "ERROR")
    sys.exit(1)

# --- Step 2: Set Environment & Execute Setup ---
os.chdir(repo_path)
log_message(f"Current working directory: {os.getcwd()}")
os.environ['PROJECT_ROOT'] = repo_path

pre_flight_script_path = os.path.join(repo_path, 'scripts', 'pre_flight_setup.py')
if not os.path.exists(pre_flight_script_path):
    log_message(f"FATAL: Pre-flight script not found at {pre_flight_script_path}", "ERROR")
    sys.exit(1)

log_message(f"Executing Pre-Flight Setup Script: {pre_flight_script_path}")
try:
    process = subprocess.Popen(
        [sys.executable, pre_flight_script_path],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True
    )
    with open(session_log_file, "a") as f:
        for line in iter(process.stdout.readline, ''):
            print(line, end='')
            f.write(line)
    if process.wait() != 0:
        log_message("Pre-flight setup script failed.", "ERROR")
        sys.exit(1)
    log_message("Pre-Flight Setup Script Finished Successfully.")
except Exception as e:
    log_message(f"FATAL: An unexpected error occurred during pre-flight setup: {e}", "ERROR")
    sys.exit(1)

log_message("--- CELL 1: PRE-FLIGHT SETUP END ---")

[2025-06-27 03:42:41] [INFO] --- CELL 1: PRE-FLIGHT SETUP (ALWAYS FORCING CLEAN CLONE) ---
[2025-06-27 03:42:41] [WARNING] Removing existing directory at /content/AnxLight to ensure a clean state...
[2025-06-27 03:42:43] [INFO] Directory removed successfully.
[2025-06-27 03:42:43] [INFO] Cloning fresh AnxLight repository to /content/AnxLight...
[2025-06-27 03:42:43] [INFO] Fresh clone successful.
[2025-06-27 03:42:43] [INFO] Current working directory: /content/AnxLight
[2025-06-27 03:42:43] [INFO] Executing Pre-Flight Setup Script: /content/AnxLight/scripts/pre_flight_setup.py
--- AnxLight Pre-Flight Setup Script v0.1.15 ---
Project Root: /content/AnxLight
Virtual Env Path: /content/AnxLight/anxlight_venv

--- Ensuring essential system packages are installed (e.g., aria2) ---

$ apt-get update -y
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos

In [ ]:
# @title Cell 2: AnxLight v3 Gradio App Launcher (No Inline Display)
# Using the patched launcher approach to avoid Colab subprocess issues

import os
import sys
from datetime import datetime
import time
import io
import contextlib

# --- Logging Setup ---
session_log_file = os.environ.get('ANXLIGHT_LOG_FILE')
if not session_log_file:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_dir = f"/content/anxlight_logs_{timestamp}"
    os.makedirs(log_dir, exist_ok=True)
    session_log_file = f"{log_dir}/session_log_{timestamp}.txt"

def log_message(message, level="INFO"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_msg = f"[{timestamp}] [{level}] {message}"
    print(formatted_msg)
    with open(session_log_file, "a") as f:
        f.write(formatted_msg + "\n")

log_message("--- CELL 2: NO INLINE DISPLAY LAUNCHER START ---")

# --- Fix the json_utils.save error first ---
def fix_json_save_error():
    try:
        with open('/content/AnxLight/scripts/main_gradio_app.py', 'r') as f:
            content = f.read()

        # Fix the incorrect save call
        if 'json_utils.save(config_path, config_data)' in content:
            content = content.replace(
                'json_utils.save(config_path, config_data)',
                'json_utils.save(config_path, "", config_data)'
            )
            with open('/content/AnxLight/scripts/main_gradio_app.py', 'w') as f:
                f.write(content)
            log_message("✅ Fixed json_utils.save() call in main_gradio_app.py")
        else:
            log_message("✅ json_utils.save() call already appears to be correct")
    except Exception as e:
        log_message(f"❌ Could not fix json_utils.save() error: {e}", "ERROR")

fix_json_save_error()

# --- Setup Environment ---
project_root = '/content/AnxLight'
os.chdir(project_root)

# Add all necessary paths to Python path
sys.path.insert(0, project_root)
sys.path.insert(0, os.path.join(project_root, 'modules'))
sys.path.insert(0, os.path.join(project_root, 'scripts'))
sys.path.insert(0, os.path.join(project_root, 'scripts', 'data'))

# Set environment variables
os.environ['PROJECT_ROOT'] = project_root
os.environ['PYTHONPATH'] = ':'.join(sys.path)

log_message("Environment setup complete. Launching Gradio app...")

try:
    # Activate the virtual environment by modifying sys.path
    venv_path = os.path.join(project_root, 'anxlight_venv')
    site_packages = os.path.join(venv_path, 'lib', f'python{sys.version_info.major}.{sys.version_info.minor}', 'site-packages')
    sys.path.insert(0, site_packages)

    log_message("Virtual environment packages loaded")

    # Direct import and launch - no subprocess
    from scripts.main_gradio_app import setup_gradio_interface

    log_message("Creating Gradio interface...")
    app = setup_gradio_interface()

    log_message("🚀 Launching Gradio (suppressing inline display)...")

    # Capture stdout during launch to prevent inline display
    captured_output = io.StringIO()

    with contextlib.redirect_stdout(captured_output):
        # Launch with quiet=True to prevent inline display
        app.launch(
            debug=False,
            share=True,
            quiet=True,            # This prevents inline display
            prevent_thread_lock=True,
            inbrowser=False        # Don't auto-open browser
        )

    # Give it a moment to start
    time.sleep(3)

    # Extract and display the URL from captured output
    output_text = captured_output.getvalue()

    # Look for the public URL in the captured output
    public_url = None
    for line in output_text.split('\n'):
        if 'Running on public URL:' in line:
            public_url = line.split('Running on public URL:')[-1].strip()
            break
        elif 'https://' in line and 'gradio.live' in line:
            # Alternative way to find the URL
            import re
            url_match = re.search(r'https://[^\s]+gradio\.live[^\s]*', line)
            if url_match:
                public_url = url_match.group()
                break

    if public_url:
        log_message("✅ Gradio server is running!")
        log_message(f"🔗 PUBLIC URL: {public_url}")
        log_message(f"📋 Click here to access AnxLight: {public_url}")
    else:
        log_message("✅ Gradio server is running!")
        log_message("🔍 Looking for public URL in output...")
        # Print the captured output to help find the URL
        if output_text.strip():
            print("Gradio launch output:")
            print(output_text)
        else:
            log_message("⚠️ No output captured - server may still be starting")

    log_message("🔄 Server monitoring active - no more inline displays!")

    # Keep alive loop with minimal logging
    start_time = time.time()
    counter = 0

    while True:
        time.sleep(60)  # Check every minute
        counter += 1

        # Log every 10 minutes to reduce noise
        if counter % 10 == 0:
            elapsed_mins = int((time.time() - start_time) / 60)
            log_message(f"⏱️ Server running for {elapsed_mins} minutes")
            if public_url:
                log_message(f"🔗 URL still active: {public_url}")

except Exception as e:
    log_message(f"❌ ERROR during Gradio launch: {e}", "ERROR")
    import traceback
    error_details = traceback.format_exc()
    log_message(f"📝 Full error traceback:\n{error_details}", "ERROR")

log_message("--- CELL 2: LAUNCHER END ---")

[2025-06-27 03:44:57] [INFO] --- CELL 2: NO INLINE DISPLAY LAUNCHER START ---
[2025-06-27 03:44:57] [INFO] ✅ Fixed json_utils.save() call in main_gradio_app.py
[2025-06-27 03:44:57] [INFO] Environment setup complete. Launching Gradio app...
[2025-06-27 03:44:57] [INFO] Virtual environment packages loaded
[2025-06-27 03:45:04] [INFO] Creating Gradio interface...
[2025-06-27 03:45:05] [INFO] 🚀 Launching Gradio (suppressing inline display)...


[2025-06-27 03:45:10] [INFO] ✅ Gradio server is running!
[2025-06-27 03:45:10] [INFO] 🔗 PUBLIC URL: https://d6ec54e8d5427641d4.gradio.live
[2025-06-27 03:45:10] [INFO] 📋 Click here to access AnxLight: https://d6ec54e8d5427641d4.gradio.live
[2025-06-27 03:45:10] [INFO] 🔄 Server monitoring active - no more inline displays!
